## WRF Test Case Study: [Major Snowstorm December 16-17, 2020](https://www.weather.gov/aly/Snow17Dec20)

###  WPS: Model Domain Initialize
Detailed workflow [here](https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/v4.4/users_guide_chap3.html) \
Summary workflow [here](https://www2.mmm.ucar.edu/wrf/OnLineTutorial/compilation_tutorial.php)	

1. Generate grid information

```{note}
Everytime `edit` is used in a command it implies you will use your chosen editing method. This could be via 'jupyterhub or terminal-based editor such as `emacs`
```

In WPS directory. Edit namelist.wps 
```
cd ../WPS
edit namelist.wps 
```

Change the following lines only to the values shown (mostly getting rid of 2nd number and a `,` at end of each line).
This locates the domain over the -79W,41E. Approximately the center of action for the storm.

```
&share
 wrf_core = 'ARW',
 max_dom = 1,
 start_date = '2020-12-15_00:00:00',
 end_date   = '2020-12-17_18:00:00',
 interval_seconds = 10800
/

&geogrid
 parent_id         =   1,   
 parent_grid_ratio =   1,   
 i_parent_start    =   1,  
 j_parent_start    =   1, 
 e_we              =  100,
 e_sn              =  80, 
 geog_data_res = 'default',
 dx = 15000,
 dy = 15000,
 map_proj = 'lambert',
 ref_lat   =  41.00,
 ref_lon   = -76.00,
 truelat1  =  30.0,
 truelat2  =  60.0,
 stand_lon = -79.0,
 geog_data_path = '/glade/work/wrfhelp/WPS_GEOG/'
/
```

Plot domain grid (requires a simple [ncl](https://www.ncl.ucar.edu) command and a module load if not currently installed))
First you will need to edit the ncl file to get the right output tyoe `.png` to be easily viewd in `jupyterhub`

```
edit util/plotgrids_new.ncl (and set `type = "png"`)
```

Now run grid domain image code

```
ncl < util/plotgrids_new.ncl
```



View created image file `wps_show_dom.png` in jupyterhub 

Generate surface data for this domain (takes a few minutes)
```
./geogrid.exe	
```
Successful if last line on screen is
```
!  Successful completion of geogrid.        !
```








2. Check generated grid data
Data should be in `geo_em.d01.nc` (check it exists)
Browse data (should for the most part be obviously North East USA)  
```
ncview geo_em.d01.nc
```
Variable descriptions very limited. Try ncdump to look at the descriptions for each variable.
```
ncdump -h geo_em.d01.nc | less
```




3. Grab domain datasets for this case study (Run WPS)

Raw grib files (GFS) used to initialize WRF forecasts can be found [here](https://rda.ucar.edu/datasets/ds084.1/). If a different time period is needed then additional data has to be processed. 
```{note} Data needed to run this case study can be found here:
    `/glade/scratch/rneale/GFS/`
```

With the changes to `namelist.wps` run WPS to set up processing of meteorology fields for initial and boundary forcing
```
./link_grib.csh /glade/scratch/rneale/GFS/asp1/
```
Then link to the correct Vtable (GFS, for this case):
```
ln -sf ungrib/Variable_Tables/Vtable.GFS Vtable
```

Then run the `ungrib` executable. Ungrib translates the raw GRIB files from GFS so tha `metgrid` can perform required interpolation to the model domain (takes a few minutes)
```{warning} 
Using `bash` shell here may lead to problems. Try switching to `csh` if that's the case.
```
```
./ungrib.exe
```
The extensive output from running this command details the available data variables and levels used for interpolation.
Look for:
```
!  Successful completion of ungrib.   !
```
`ungrib` produces a number of local files e.g. `FILE:2020-12-15_00` which are prestaged for WRF to be run.

```{warning}
If you are to rerun this step you must delete all the `FILE:*` files first

```

Finally for data preparation for WRF is interpolation performed in `metgrid` (takes about a minute).
```
./metgrid.exe >& log.metgrid
```



4. Running the model

Change to WRF directory from WPS directory
```
cd ../WRF
```
Let’s run the case here
```
cd run (we can also run it here: test/em_real)
```
And similar to linking the files we generated with WPS do the following
```
ln -sf ../../WPS/met_em.d01.2020-12* .
```
or
```
ln -sf ../../../WPS/met_em.d01.2020-12* .
```
Look over namelist, make sure it reflects the period of the
```	
edit namelist.input 
```
We are only using the first column of values as we only have a single domain and no nesting.
Make sure you change:
*Trun off second domain `input_from_file=.true.,.false.`
*Dates of forecast should be `'2020-12-15_00:00:00' to '2020-12-17_18:00:00'`
*Domain range: `e_we=100` and `e_sn=80`
*Gravity wave drag option to `gwd_opt=0` instead	

You can now run the `real` program, which processe data for a real case study, on a compute node
```
qcmd -- ./real.exe
```

RUN WRF (on a compute node)!

```
qcmd -- ./wrf.exe
```

You will find all the output from the run in this directory
```
cd ../test/em_real/
```
You should see hourly `wrfout` files as the model runs from `2020-12-15` through `2020-12-17` similar to
```
wrfout_d01_2020-12-15_00:00:00
```

When run on a compute node the simulations hopefully takes around 30 minutes to complete

5. Simple analysis of the output

OK let's take a look inside these with one of the notebooks.
Make a new `diagnostics` directory
```
mkdir /glade/work/$USER/ASP2023/diags
```
Copy across the set of notebooks for simple analysis

```
cp /glade/work/rneale/ASP2023/diags/* /glade/work/$USER/ASP2023/diags
```



